##Read in data, apply indexs and summarise for the first time

In [20]:
#Read in data
AllData=pd.read_csv('K:\Rainshelter\StonySoilLysimeters\DownloadedData\StonyLysimetersCS650.dat', #specify file path for data to read in
                         parse_dates=True, #tell the function to parse date columns to datetime formats
                         dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                         skiprows = [0,2,3], #leave out rows 1, 3 and 4 which have redundant information
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')

#Read in calibration coefficients calculated in C:\GitHubRepos\StonySoilLysimeters\Calibration\CS650Calibration.ipynb
SensorCoefficients = pd.read_pickle('C:\GitHubRepos\StonySoilLysimeters\Calibration\CS650Calibration.pkl')

#Bring in all index data
AllDataIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\StonyLysimetersWater&TempIndex.csv',
                         index_col = 0)

#Bring in index for horizion types
HorizonIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\HorizonIndex.csv', #specify file path for data to read in
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')

#Apply indexes to data
AllDataTransposed = AllData.transpose()
AllDataIndexed = pd.concat([AllDataIndex,AllDataTransposed], axis=1)
AllDataIndexed.index.name='ColumnHeader'
AllDataIndexed.set_index(['Measurement','Depth','Gravels','Stones','Column','Sensor', 'MUX', 'Port','Units','Summary','Block','Treatment'], 
                        append=False, inplace=True)
AllDataIndexed.sort(inplace=True)
LysoData=AllDataIndexed.transpose()
LysoData.index = LysoData.index.to_datetime()  ## for some reason the concat function changes the data type on the date indes so need to change it back

##Apply calibrations
VWC_uncalibrated = LysoData.VolumetricWaterContent
VWC_uncalibrated.columns = VWC_uncalibrated.columns.swaplevel('Depth','Sensor') #Make sensor the top level so index selection works properly
VWC = pd.DataFrame(index = VWC_uncalibrated.index, columns = VWC_uncalibrated.columns)  #Set up empty dataframe with the same index and column structure as TDR data

for X in range (1,97): 
    SensorRef = 'S_' + np.str(X)
    Horizon = HorizonIndex.ix[X,'Horizon']
    Slope = SensorCoefficients.ix[Horizon,'Slope']
    Intercept = SensorCoefficients.ix[Horizon,'Intercept']
    VWC[SensorRef] = VWC_uncalibrated.ix[:,SensorRef] * Slope + Intercept 

#Set up status files
LastRow = VWC.index.size
np.save('LastRow.npy',LastRow)
VWC.to_pickle('.\DataFrames\LysoVWC_df')

##Bring in libraries

In [1]:
import datetime
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.tools as tls
import time 
from plotly.graph_objs import *
Tokens = tls.get_credentials_file()['stream_ids']

##Function to read in fresh data and update status files

In [2]:
def UpdataDataFrame():
    #Bring in previous status files
    LastRead = np.load('.\LastRow.npy')
    VWC = pd.read_pickle('.\DataFrames\LysoVWC_df')
    
    #Bring in fresh data
    StartRead = LastRead + 4
    Skips = [0,2,3] + range(4,StartRead)
    FreshData=pd.read_csv('K:\Rainshelter\StonySoilLysimeters\DownloadedData\StonyLysimetersCS650.dat', #specify file path for data to read in
                             parse_dates=True, #tell the function to parse date columns to datetime formats
                             dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                             skiprows = Skips, #leave out rows 1, 3 and 4 which have redundant information
                             index_col = 0, #Use the first column, which is Date, as an index
                             na_values = 'NAN')

    #Read in calibration coefficients calculated in C:\GitHubRepos\StonySoilLysimeters\Calibration\CS650Calibration.ipynb
    SensorCoefficients = pd.read_pickle('C:\GitHubRepos\StonySoilLysimeters\Calibration\CS650Calibration.pkl')

    #Bring in all index data
    AllDataIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\StonyLysimetersWater&TempIndex.csv',
                             index_col = 0)

    #Bring in index for horizion types
    HorizonIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\HorizonIndex.csv', #specify file path for data to read in
                             index_col = 0, #Use the first column, which is Date, as an index
                             na_values = 'NAN')

    #Apply indexes to data
    FreshDataTransposed = FreshData.transpose()
    FreshDataIndexed = pd.concat([AllDataIndex,FreshDataTransposed], axis=1)
    FreshDataIndexed.index.name='ColumnHeader'
    FreshDataIndexed.set_index(['Measurement','Depth','Gravels','Stones','Column','Sensor', 'MUX', 'Port','Units','Summary','Block','Treatment'], 
                            append=False, inplace=True)
    FreshDataIndexed.sort_index(inplace=True)
    FreshLysoData=FreshDataIndexed.transpose()
    FreshLysoData.index = FreshLysoData.index.to_datetime()  ## for some reason the concat function changes the data type on the date indes so need to change it back

    ##Apply calibrations
    FreshVWC_uncalibrated = FreshLysoData.VolumetricWaterContent
    FreshVWC_uncalibrated.columns = FreshVWC_uncalibrated.columns.swaplevel('Depth','Sensor') #Make sensor the top level so index selection works properly
    FreshVWC = pd.DataFrame(index = FreshVWC_uncalibrated.index, columns = FreshVWC_uncalibrated.columns)  #Set up empty dataframe with the same index and column structure as TDR data

    for X in range (1,97): 
        SensorRef = 'S_' + np.str(X)
        Horizon = HorizonIndex.ix[X,'Horizon']
        Slope = SensorCoefficients.ix[Horizon,'Slope']
        Intercept = SensorCoefficients.ix[Horizon,'Intercept']
        FreshVWC[SensorRef] = FreshVWC_uncalibrated.ix[:,SensorRef] * Slope + Intercept 

    #Join fresh data onto existing data
    VWC = pd.concat([VWC,FreshVWC])
    VWC.sort_index(inplace = True)
    
    #Update status files
    LastRow = VWC.index.size
    np.save('LastRow.npy',LastRow)
    VWC.to_pickle('.\DataFrames\LysoVWC_df')
    return VWC

##Set up treatment description classes

In [3]:
#Treatment class constructor
class Treatment(object):
    def __init__(self, Number, Depth, Stonyness):
        self.Number=Number
        self.Depth=Depth
        self.Stonyness=Stonyness
        self.streamID=Tokens[number+6] #First 6 takens used by Rainshelter graph, lysimeter uses the 2nd set
        self.Label= Depth + ' ' + Stonyness
        self.Trace = Scatter(x=[],y=[], name = self.Label,
                stream = Stream(token=self.streamID)
                )
        self.Stream = py.Stream(self.streamID)

#Creat class with treatment data
Treatments = []
Colors = ['red','blue','green']
Depths = ['Deep', 'Shallow']
Stones = ['StoneFree', 'Stony', 'VeryStony']
Lines = ['solid','dash']
number = 0
LinePos = 0
for D in Depths:
    ColorPos = 0
    for S in Stones:
        Treatments.append(Treatment(number, D, S))
        Treatments[number].Trace.line = Line(color = Colors[ColorPos], dash = Lines[LinePos])
        ColorPos +=1
        number += 1
    LinePos+=1

##Build base graph

In [4]:
GraphData = Data([Treatments[0].Trace,
                  Treatments[1].Trace,
                  Treatments[2].Trace,
                  Treatments[3].Trace,
                  Treatments[4].Trace,
                  Treatments[5].Trace
                 ])

py.iplot(GraphData, filename = 'LysimeterSWC')

##Function to update plotly graph

In [5]:
def Update():
    #Update Data Frame with data that has been logged since last update
    FieldData = UpdataDataFrame()
    
    #Calculate treatment means omitting data prior to 2014-11-05 08:00:00 to avoid NaN values
    DataMeans =  FieldData.ix['2015-07-01 08:00:00':].groupby(level=['Depth','Gravels','Stones'],axis=1).mean()
    DataMeans =  DataMeans.dropna(axis=1) #For some reason it keeps non valid combinations in so need to extract with this function
    DataMeans.where(DataMeans<0.7,np.nan,inplace=True)

    
    #Calculate the water content of the soil profile by multiplying the volumetric water content by each layers
    #depth and summing.  The 0-15 layers are divided by 2 to average the two readings
    ProfileWater = DataMeans.ix[:,'D1'] * 150 + \
                   DataMeans.ix[:,'D2'].values * 150 + \
                   DataMeans.ix[:,'D3'].values * 150 + \
                   DataMeans.ix[:,'D4'].values * 150 
    
    FieldCapacity = ProfileWater.resample('D', how='max')
    FieldCapacity = FieldCapacity.ix['2015-07-11'] # I would have though this would return a data frame with a single row but instead it returns a series with a multiindex in columns
    SoilWaterDeficit = -(FieldCapacity - ProfileWater)  # This calculation only works because field capacity is a multi index series

    X = SoilWaterDeficit.index
    for Treatment in Treatments:
        Treatment.Stream.open()
        Treatment.Stream.write({'x':X,'y':SoilWaterDeficit.ix[:,(Treatment.Depth,Treatment.Stonyness)]})
        Treatment.Stream.close()

##One off update

In [6]:
Update()

##Continious update

In [ ]:
while True:
    Update()
    time.sleep(3600)